# Newport Bicycle and Pedestrian Advisory Commission 2014 Survey
This notebook is provided as a starting point for working with the [BPAC 2014 survey dataset](https://github.com/newportdataportal/newport-crosswalk-survey-2014).  We'll use the [Bokeh](https://bokeh.pydata.org/en/latest/) library to visualize the data.

In [1]:
import pandas as pd
import geopandas as gpd
import fiona
import requests

from bokeh.plotting import figure, output_notebook, show, output_file
from bokeh.models import GeoJSONDataSource, ColorBar, HoverTool, Range1d
from bokeh.tile_providers import STAMEN_TONER, CARTODBPOSITRON
from bokeh.models.mappers import LinearColorMapper
from bokeh.palettes import Spectral6

In [2]:
# load bokeh for inline plotting
output_notebook()

Loading BokehJS ...

Load the dataset from the data portal and prepare it for plotting.

In [3]:
# get the data from the portal
survey_response = requests.get("https://raw.githubusercontent.com/NewportDataPortal/newport-crosswalk-survey-2014/master/bpac_survey_2014.geojson")

b = bytes(survey_response.content)
with fiona.BytesCollection(b) as f:
    crs = f.crs
    survey_gdf = gpd.GeoDataFrame.from_features(f, crs=crs)

survey_gdf

,I don't know.,Safe,Unsafe,geometry,id,n,score,size
0,112,17,114,POINT (-71.31159975783214 41.50792163382231),1,131,0.1297709923664122,14.30131004366812
1,140,57,47,POINT (-71.30795507872978 41.5075726847703),2,104,0.5480769230769231,11.35371179039301
2,110,38,92,POINT (-71.30169157668472 41.50598751268677),3,130,0.2923076923076923,14.19213973799127
3,74,70,96,POINT (-71.30710260670982 41.49865626332756),4,166,0.4216867469879518,18.12227074235808
4,34,69,136,POINT (-71.30935118227099 41.493963557599),5,205,0.3365853658536586,22.37991266375546
5,13,136,92,POINT (-71.31173160202168 41.49181256414439),6,228,0.5964912280701754,24.89082969432314
6,38,108,91,POINT (-71.31209281897929 41.49148517397211),7,199,0.542713567839196,21.72489082969432
7,33,97,109,POINT (-71.31267076611148 41.48954650321857),8,206,0.470873786407767,22.48908296943231
8,47,75,114,POINT (-71.30688045828089 41.48339865277515),9,189,0.3968253968253968,20.63318777292577
9,8,113,115,POINT (-71.30866848222109 41.4837179657074),10,228,0.4956140350877193,24.89082969432314


In [4]:
# define the bokeh datasource as a geosource.  Note the coordinate reference system is 3857
geosource = GeoJSONDataSource(geojson=survey_gdf.to_crs(epsg=3857).to_json())

Build a Bokeh plot.  Include tooltips and a color scale for the safe/unsafe score.

In [5]:
# define tooltip information
hover = HoverTool(tooltips=[
    ("Intersection", "@id"),
    ("Safe", "@Safe"),
    ("Unsafe", "@Unsafe"),
    ("Score", "@score"),
])

# define color map for plot, swapped low-high for numbers closer to 1 to be green
cmap = LinearColorMapper(palette = Spectral6, low=1, high=0)

# set plotting ranges based on the extent of the geospatial dataset
minx, miny, maxx, maxy = survey_gdf.to_crs(epsg=3857).total_bounds
mercator_x_extent = dict(start=minx-25, end=maxx+25, bounds=None)
mercator_y_extent = dict(start=miny-25, end=maxy+25, bounds=None)
x_range = Range1d(**mercator_x_extent)
y_range = Range1d(**mercator_y_extent)

# build plot
p = figure(tools=["wheel_zoom,box_zoom,pan,save", hover], x_range=x_range, y_range=y_range)
p.add_tile(CARTODBPOSITRON)
p.circle(x='x', y='y', size='size', alpha=0.9, source=geosource, fill_color={'field': 'score', 'transform':cmap}, line_color=None)
show(p)

### Look at the raw survey responses
The data above was processed from the raw survey responses.  Here they are:

In [6]:
raw_survey = pd.read_csv("https://raw.githubusercontent.com/NewportDataPortal/newport-crosswalk-survey-2014/master/Newport%20Pedestrian%20Safety%20Survey%20-%20May%202014%20cleaned.csv")
raw_survey.head()

,Respondent ID,Collector ID,Start Date,End Date,1) Adm Kalbfus at Girard (Mainstay/Newport Grand) - Safe,1) Adm Kalbfus at Girard (Mainstay/Newport Grand) - Unsafe,1) Adm Kalbfus at Girard (Mainstay/Newport Grand) - I don't know.,2) Adm Kalbfus at Hillside - Safe,2) Adm Kalbfus at Hillside - Unsafe,2) Adm Kalbfus at Hillside - I don't know.,...,23) Thames St at Wellington - Safe,23) Thames St at Wellington - Unsafe,23) Thames St at Wellington - I don't know.,24) Thames St at Morton Ave - Safe,24) Thames St at Morton Ave - Unsafe,24) Thames St at Morton Ave - I don't know.,25) Carroll Avenue at Harrison (St. Augustin's) - Safe,25) Carroll Avenue at Harrison (St. Augustin's) - Unsafe,25) Carroll Avenue at Harrison (St. Augustin's) - I don't know.,Comment/Explanation
0,4692619912,52541754,4/29/2016 2:41,4/29/2016 2:42,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4680235760,52541754,4/24/2016 11:51,4/24/2016 11:56,NaN,Unsafe,NaN,Safe,NaN,NaN,...,Safe,NaN,NaN,Safe,NaN,NaN,Safe,NaN,NaN,Broadway at Miantonomi/Adm Kalbfus Rd is in M...
2,3428074483,52541754,8/27/2014 8:36,8/27/2014 8:44,NaN,Unsafe,NaN,NaN,Unsafe,NaN,...,NaN,Unsafe,NaN,NaN,Unsafe,NaN,NaN,Unsafe,NaN,NaN
3,3376193409,52541754,7/25/2014 17:30,7/25/2014 17:57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3372701295,52541754,7/23/2014 20:29,7/23/2014 20:53,NaN,NaN,I don't know.,NaN,NaN,I don't know.,...,Safe,NaN,NaN,Safe,NaN,NaN,Safe,NaN,NaN,NaN
